In [7]:
import pandas as pd
import os
from scipy import io

import warnings
warnings.filterwarnings(action="ignore")

In [8]:
dataset_dir = 'ARS DLR Data Set/'
os.listdir(dataset_dir)

['ARS_DLR_Benchmark_Data_Set_Readme.txt',
 'ARS_DLR_DataSet_V2.mat',
 'ARS_DLR_Benchmark_Data_Set.mat',
 'QuantizationFiles.zip',
 'ARS_DLR_DataSet.mat',
 'ARS_DLR_Data_Set_Readme.txt',
 'ARS_DLR_Data_Set_V2_Readme.txt',
 'Subjects_Age_Height_Gender.xls']

In [9]:
per_info = pd.read_excel(dataset_dir + 'Subjects_Age_Height_Gender.xls')
per_info.head()

,Name,Age,Height,Gender,Unnamed: 4,Unnamed: 5
0,Marina,24,1.66,Female,NaN,NaN
1,Maria,25,1.61,Female,NaN,NaN
2,Elena,23,1.7,Female,NaN,NaN
3,Cristina,27,1.6,Female,NaN,NaN
4,Susanna,41,1.61,Female,NaN,NaN


In [10]:
columns = ['time', 'acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'mag_x', 'mag_y', 'mag_z']
dataset_list = ['ARS_DLR_Benchmark_Data_Set.mat', 'ARS_DLR_DataSet.mat', 'ARS_DLR_DataSet_V2.mat']
data_len = []

save_dir = 'dlr_preprocessed/'
train_dir, valid_dir, test_dir = 'train/', 'valid/', 'test/'
for dir_name in [train_dir, valid_dir, test_dir]:
    if not os.path.exists(save_dir + dir_name):
        os.makedirs(save_dir + dir_name, exist_ok=True)

In [11]:
def save_act(name, act_data, index=0):
    if name in ['Maria', 'Matthias', 'Susanna', 'Marina', 'Hanno', 'Emil', 'Korbinian', 'Jesus', 'Fabian']:
        act_data.to_csv(f"{save_dir + train_dir + key}_{label[i].item()}_{index}.csv")
    elif name in ['Cristina', 'Mohammed', 'Paula']:
        act_data.to_csv(f"{save_dir + valid_dir + key}_{label[i].item()}_{index}.csv")
    else:
        act_data.to_csv(f"{save_dir + test_dir + key}_{label[i].item()}_{index}.csv")

In [12]:
for dataset in dataset_list:
    matfile = io.loadmat(dataset_dir + dataset)
    for key, val in matfile.items():
        if key.startswith('ARS'):
            name = key.split('_')[1]
            if name == "Michael":
                continue
            
            label = val[0][2][0]
            label_idx = val[0][3][0]
    
            age = per_info[per_info['Name'] == name]['Age'].values[0]
            gender = per_info[per_info['Name'] == name]['Gender'].values[0]
            height = per_info[per_info['Name'] == name]['Height'].values[0]
            
            sensor_data = pd.DataFrame(val[0][0], columns=columns)
            sensor_data['age'] = age
            sensor_data['gender'] = gender
            sensor_data['height'] = height
            
            for i in range(len(label)):
                start = label_idx[i * 2] - 1
                end = label_idx[(i * 2) + 1] - 1
                if label[i].item() in ['JUMPBCK', 'JUMPFWD', 'JUMPVRT', 'WALKUPS', 'WALKDWS', 'TRANSUP', 'JUMPING']:
                    continue
                if end - start > 2000:
                    div = (end - start) // 2000
                    for j in range(div + 1):
                        if start + ((j + 1) * 2000) > end:
                            act_data = sensor_data.loc[start + (j * 2000): end]
                        else:
                            act_data = sensor_data.loc[start + (j * 2000): start + ((j + 1) * 2000)]
                        act_data['label'] = label[i].item()
                        if len(act_data) > 500:
                            if act_data.isna().any().any():
                                print(act_data)
                            save_act(name, act_data, j)
                    
                elif end - start > 500:
                    act_data = sensor_data.loc[start: end]
                    act_data['label'] = label[i].item()
                    if act_data.isna().any().any():
                        print(act_data)
                    save_act(name, act_data, index=0)